In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Read data

In [2]:
muni = pd.read_csv('/home/juliane.oliveira/Documents/AESOP data structure/ETL_data/DTB_BRASIL_MUNICIPIO.csv',sep=';')


In [3]:
link = '/opt/storage/trusted/aesop/visualization/Mobility/fluvi_road_ibge.parquet'

df = pd.read_parquet(link, engine='pyarrow')

In [4]:
df.head()

,ID,co_uf_ori,uf_ori,co_muni_ori,nome_muni_ori,co_uf_des,uf_des,co_muni_des,nome_muni_des,fluv_conec,road_conec,tot_conec,long_muni_ori,lat_muni_ori,long_muni_des,lat_muni_des,irregular_conec
0,13026031304260,13,AM,1302603,Manaus,13,AM,1304260,Uarini,7.5,0.0,7.5,-60.023335,-3.134691,-65.155346,-2.983609,0.0
1,13021081302603,13,AM,1302108,Japurá,13,AM,1302603,Manaus,3.0,0.0,3.0,-66.996903,-1.880845,-60.023335,-3.134691,3.0
2,51038095108402,51,MT,5103809,Figueirópolis D'Oeste,51,MT,5108402,Várzea Grande,0.0,42.0,42.0,-58.737825,-15.446135,-56.144664,-15.658065,0.0
3,13000291302603,13,AM,1300029,Alvarães,13,AM,1302603,Manaus,7.5,0.0,7.5,-64.813535,-3.216584,-60.023335,-3.134691,0.0
4,13000291301308,13,AM,1300029,Alvarães,13,AM,1301308,Codajás,6.0,0.0,6.0,-64.813535,-3.216584,-62.063430,-3.843046,0.0


In [5]:
df = df.assign(tot_pass = df.tot_conec*25)

# Format IBGE data for analysis

In [10]:
set_lis = ['AM', 'MT', 'PA', 'CE', 'PI', 'RN', 'SE', 'MG', 'BA', 'SP', 'PR',
       'MA', 'AL', 'RS', 'SC', 'PE', 'AP', 'RJ', 'TO', 'GO', 'RO', 'PB',
       'ES', 'AC', 'RR', 'MS', 'DF']

df = df[df.uf_des.isin(set_lis)]

In [11]:
data = df.pivot(index='co_muni_ori', columns='co_muni_des', values='tot_pass')

In [13]:
lst = set(muni['Código Município Completo']) - set(data.columns)

In [14]:
for value in lst:
    
    lis = [0]*len(data)

    d = {value: lis}
    dtf = pd.DataFrame(data=d)
    dtf.set_index(data.index)
    
    data = data.join(dtf)

In [15]:
lst_index = set(muni['Código Município Completo']) - set(data.index)

In [16]:
for value in lst_index:
    data.loc[value] = [0]*5570

In [17]:
data = data.fillna(0)
data

,1100023,1100049,1100056,1100064,1100072,1100080,1100098,1100106,1100114,1100122,...,3135456,2500577,4106209,2201572,4218855,3102704,2500601,5203962,3115003,3528700
co_muni_ori,,,,,,,,,,,,,,,,,,,,,
1100015,175.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,175.0,350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1100023,0.0,4925.0,700.0,700.0,0.0,675.0,150.0,0.0,5875.0,8537.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1100031,0.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1100049,0.0,0.0,525.0,525.0,0.0,0.0,1225.0,0.0,4325.0,4975.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1100056,0.0,0.0,0.0,1400.0,600.0,0.0,0.0,0.0,525.0,700.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3153905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5203962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
data = data.reindex(sorted(data.columns), axis=1)

In [19]:
data = data.sort_index(axis=0)

In [20]:
final = data + data.transpose()

In [21]:
final

,1100015,1100023,1100031,1100049,1100056,1100064,1100072,1100080,1100098,1100106,...,5221601,5221700,5221809,5221858,5221908,5222005,5222054,5222203,5222302,5300108
co_muni_ori,,,,,,,,,,,,,,,,,,,,,
1100015,0.0,175.0,0.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1100023,175.0,0.0,0.0,4925.0,700.0,700.0,0.0,675.0,150.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0
1100031,0.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1100049,500.0,4925.0,0.0,0.0,525.0,525.0,0.0,0.0,1225.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700.0
1100056,0.0,700.0,0.0,525.0,0.0,1400.0,600.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1050.0,275.0,0.0,0.0,0.0,0.0,0.0,875.0
5222054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5222203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
teste = final.melt()

In [23]:
teste.head()

,variable,value
0,1100015,0.0
1,1100015,175.0
2,1100015,0.0
3,1100015,500.0
4,1100015,0.0


In [24]:
teste = teste.rename(columns={"variable":"co_muni_ori", "value":"tot_pass"})

In [25]:
lst_city = []
for city in teste.co_muni_ori.unique():
    set_muni = teste[teste.co_muni_ori == city]
    
    set_muni = set_muni.assign(co_muni_des = final.columns)
    
    set_muni = set_muni.assign(co_uf_ori = set_muni.co_muni_ori.astype(str).str[:2])
    
    set_muni = set_muni.assign(co_uf_des = set_muni.co_muni_des.astype(str).str[:2])
    
    set_muni = set_muni.assign(uf_ori = set_muni.co_uf_ori)
    
    set_muni = set_muni.replace({'uf_ori': {'13': 'AM', '11':'RO', '51':'MT', '15':'PA',
                                        '23':'CE', '22':'PI', '24':'RN', '28': 'SE',
                                        '31':'MG', '29': 'BA','35':'SP','41': 'PR',
                                        '21':'MA','27':'AL','43': 'RS','42': 'SC',
                                        '26': 'PE','16': 'AP','33': 'RJ','17': 'TO',
                                        '52': 'GO','11': 'RO','25': 'PB','32': 'ES',
                                        '12': 'AC','14': 'RR','50': 'MS', '53':'DF'}})
    
    set_muni = set_muni.assign(uf_des = set_muni.co_uf_des)
    
    set_muni = set_muni.replace({'uf_des': {'13': 'AM', '11':'RO', '51':'MT', '15':'PA',
                                        '23':'CE', '22':'PI', '24':'RN', '28': 'SE',
                                        '31':'MG', '29': 'BA','35':'SP','41': 'PR',
                                        '21':'MA','27':'AL','43': 'RS','42': 'SC',
                                        '26': 'PE','16': 'AP','33': 'RJ','17': 'TO',
                                        '52': 'GO','11': 'RO','25': 'PB','32': 'ES',
                                        '12': 'AC','14': 'RR','50': 'MS','53':'DF'}})
    
    lst_city.append(set_muni)

In [26]:
df_fin = df = pd.concat(lst_city)

# Descriptive analysis

In [27]:
df_fin.head()

,co_muni_ori,tot_pass,co_muni_des,co_uf_ori,co_uf_des,uf_ori,uf_des
0,1100015,0.0,1100015,11,11,RO,RO
1,1100015,175.0,1100023,11,11,RO,RO
2,1100015,0.0,1100031,11,11,RO,RO
3,1100015,500.0,1100049,11,11,RO,RO
4,1100015,0.0,1100056,11,11,RO,RO


In [28]:
df_fin.tot_pass.sum()

157222962.5

In [29]:
round(df_fin.tot_pass.sum()*100/203062512,1)

77.4

# Grafico de cordas

In [30]:
data_chord = df_fin.groupby(['uf_ori','uf_des'])['tot_pass'].sum().reset_index()

In [31]:
data_chord

,uf_ori,uf_des,tot_pass
0,AC,AC,196650.0
1,AC,AL,0.0
2,AC,AM,3500.0
3,AC,AP,0.0
4,AC,BA,0.0
...,...,...,...
724,TO,RS,24900.0
725,TO,SC,11725.0
726,TO,SE,750.0
727,TO,SP,19150.0


# State level analysis

In [33]:
data_chord = data_chord.rename(columns={"uf_ori":"source", "uf_des":"target", "tot_pass":"weight"})
data_chord.head()

,source,target,weight
0,AC,AC,196650.0
1,AC,AL,0.0
2,AC,AM,3500.0
3,AC,AP,0.0
4,AC,BA,0.0


In [36]:
data_chord2 = data_chord[data_chord['source'] != data_chord['target']]

In [35]:
#pip install d3blocks

# Load d3blocks
from d3blocks import D3Blocks

# Initialize
d3 = D3Blocks()

# Load example data
df1 = data_chord2

#data_chord[(data_chord.target != 'SP') & (data_chord.source != 'SP')]
#[data_chord.ano == '2019'][['source','target','weight']] 

# Plot
d3.chord(df1)

# Or specify the output path
d3.chord(df1, filepath='/home/juliane.oliveira/workspace/Reproducibility-results_paper2/images/chord_ufs_ibge.html')

[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Initializing [Chord]
[d3blocks] >INFO> filepath is set to [/tmp/d3blocks/chord.html]
[d3blocks] >INFO> Convert to Frame.
[d3blocks] >INFO> Node properties are set.
[d3blocks] >INFO> Set edge-opacity based on the [source] node-opacity.
[d3blocks] >INFO> Set edge-colors based on the [source] node-color.
[d3blocks] >INFO> Edge properties are set.
[d3blocks] >INFO> File already exists and will be overwritten: [/tmp/d3blocks/chord.html]
[d3blocks] >INFO> Open browser: /tmp/d3blocks/chord.html
[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Initializing [Chord]
[d3blocks] >INFO> filepath is set to [/home/juliane.oliveira/workspace/Reproducibility-results_paper2/images/chord_ufs_ibge.html]
[d3blocks] >INFO> Convert to Frame.
[d3blocks] >INFO> Node properties are set.
[d3blocks] >INFO> Set edge-opacity based

# Region level analysis

In [37]:
data_chord.head()

,source,target,weight
0,AC,AC,196650.0
1,AC,AL,0.0
2,AC,AM,3500.0
3,AC,AP,0.0
4,AC,BA,0.0


In [38]:
data_chord3 = data_chord.rename(columns={"source":"source1", "target":"target1"})
data_chord3.head()

,source1,target1,weight
0,AC,AC,196650.0
1,AC,AL,0.0
2,AC,AM,3500.0
3,AC,AP,0.0
4,AC,BA,0.0


In [39]:
data_chord3 = data_chord3.assign(target = data_chord3.target1)
data_chord3 = data_chord3.assign(source = data_chord3.source1)

In [40]:
data_chord3 = data_chord3.replace({'target': {'AC': 'North', 'AP': 'North', 'AM': 'North', 'PA': 'North','TO': 'North',
                               'RO':'North', 'RR':'North', 'AL':'Northeast', 'BA':'Northeast', 'CE':'Northeast',
                               'MA': 'Northeast', 'PB': 'Northeast', 'PE': 'Northeast', 'PI': 'Northeast',
                               'RN': 'Northeast', 'SE': 'Northeast','DF': 'Midwest','GO': 'Midwest',
                               'MT': 'Midwest', 'MS': 'Midwest', 'ES': 'Southeast', 'MG': 'Southeast',
                               'RJ': 'Southeast', 'SP': 'Southeast', 'PR': 'South', 'RS':'South','SC':'South'}})


In [41]:
data_chord3 = data_chord3.replace({'source': {'AC': 'North', 'AP': 'North', 'AM': 'North', 'PA': 'North','TO': 'North',
                               'RO':'North', 'RR':'North', 'AL':'Northeast', 'BA':'Northeast', 'CE':'Northeast',
                               'MA': 'Northeast', 'PB': 'Northeast', 'PE': 'Northeast', 'PI': 'Northeast',
                               'RN': 'Northeast', 'SE': 'Northeast','DF': 'Midwest','GO': 'Midwest',
                               'MT': 'Midwest', 'MS': 'Midwest', 'ES': 'Southeast', 'MG': 'Southeast',
                               'RJ': 'Southeast', 'SP': 'Southeast', 'PR': 'South', 'RS':'South','SC':'South'}})


In [42]:
data_chord3

,source1,target1,weight,target,source
0,AC,AC,196650.0,North,North
1,AC,AL,0.0,Northeast,North
2,AC,AM,3500.0,North,North
3,AC,AP,0.0,North,North
4,AC,BA,0.0,Northeast,North
...,...,...,...,...,...
724,TO,RS,24900.0,South,North
725,TO,SC,11725.0,South,North
726,TO,SE,750.0,Northeast,North
727,TO,SP,19150.0,Southeast,North


In [43]:
region = data_chord3.groupby(['target', 'source'])['weight'].sum().reset_index()

region = region.assign(per = round(region.weight*100/sum(region.weight),1))

In [44]:
region.head()

,target,source,weight,per
0,Midwest,Midwest,8945100.00,5.7
1,Midwest,North,798343.75,0.5
2,Midwest,Northeast,523168.75,0.3
3,Midwest,South,956200.00,0.6
4,Midwest,Southeast,847693.75,0.5


In [49]:
region.groupby(['target'])['weight','per'].sum().reset_index()

/tmp/ipykernel_150740/2862005695.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  region.groupby(['target'])['weight','per'].sum().reset_index()


,target,weight,per
0,Midwest,12070506.25,7.6
1,North,9601937.50,6.1
2,Northeast,63303012.50,40.2
3,South,26688906.25,16.9
4,Southeast,45558600.00,28.9


In [50]:
region.groupby(['source'])['weight','per'].sum().reset_index()

/tmp/ipykernel_150740/1445649924.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  region.groupby(['source'])['weight','per'].sum().reset_index()


,source,weight,per
0,Midwest,12070506.25,7.6
1,North,9601937.50,6.1
2,Northeast,63303012.50,40.2
3,South,26688906.25,16.9
4,Southeast,45558600.00,28.9


In [47]:
region.head()

,target,source,weight,per
0,Midwest,Midwest,8945100.00,5.7
1,Midwest,North,798343.75,0.5
2,Midwest,Northeast,523168.75,0.3
3,Midwest,South,956200.00,0.6
4,Midwest,Southeast,847693.75,0.5


In [51]:
# Install
#pip install d3blocks

# Load d3blocks
from d3blocks import D3Blocks

# Initialize
d3 = D3Blocks()

# Load example data
#df1 = region[['target', 'source','weight']]

df1 = region[region['source'] != region['target']]

#data_chord[(data_chord.target != 'SP') & (data_chord.source != 'SP')]
#[data_chord.ano == '2019'][['source','target','weight']] 

# Plot
d3.chord(df1)

# Or specify the output path
d3.chord(df1, filepath='/home/juliane.oliveira/workspace/Reproducibility-results_paper2/images/chord_regions_ibge2.html')

[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Initializing [Chord]
[d3blocks] >INFO> filepath is set to [/tmp/d3blocks/chord.html]
[d3blocks] >INFO> Convert to Frame.
[d3blocks] >INFO> Node properties are set.
/home/juliane.oliveira/.local/lib/python3.9/site-packages/d3blocks/utils.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if 'weight' in X.columns: X['weight'] = X['weight'].astype(float)
[d3blocks] >INFO> Set edge-opacity based on the [source] node-opacity.
[d3blocks] >INFO> Set edge-colors based on the [source] node-color.
[d3blocks] >INFO> Edge properties are set.
[d3blocks] >INFO> File already exists and will be overwritten: [/

# Road and waterway analysis